In [24]:
import re
import unicodedata
import numpy as np
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt


In [25]:
def importancia(W: np.array) -> np.array:
    I = np.zeros(W.shape[1])  # Vetor de importância
    for j in range(W.shape[1]):
        soma = np.sum(np.abs(W[:, j]))
        I[j] = np.sqrt(soma)
    return I

def c(V: list, W: np.array, I: np.array) -> float:
    c_value = 0  # Valor de c
    for j in range(W.shape[1]):
        if any(W[i, j] > 0 for i in V):  # Se a característica j é relevante
            c_value += I[j]
    return c_value

def guloso(W: np.array, I: np.array, B: int) -> list:
    nao_selecionados = list(range(W.shape[0]))  # Lista de índices não selecionados
    V = []  # Conjunto de características selecionadas
    itens = 0  # Número de elementos selecionados
    c_value = 0  # Valor de c
    
    while itens < B and nao_selecionados:
        best_gain = -np.inf  # Valor de ganho máximo
        best_item = None  # Índice do melhor item 
        
        for item in nao_selecionados:  # Itera sobre os itens não selecionados
            lista_temp = V + [item]
            gain = c(lista_temp, W, I) - c(V, W, I)
            if gain > best_gain:
                best_gain = gain  # Atualiza o valor de ganho máximo
                best_item = item  # Atualiza o melhor item
        
        if best_item is not None:
            V.append(best_item)  # Adiciona o melhor item ao conjunto
            nao_selecionados.remove(best_item)  # Remove o melhor item dos não selecionados
            itens += 1
            c_value += best_gain
    
    return V

In [26]:
# Exemplo de uso
W = np.array([[1, 2, 1], [-1, 0, 2], [-2, 2, -2]])  # Exemplo de matriz W
I = importancia(W)
print("Matriz de importância: ", I)

V = [0, 1]  # Lista de índices das instâncias selecionadas
print("Cobertura: ", c(V, W, I))

V_linha = guloso(W, I, 2)
print("Melhor conjunto: ", V_linha)

Matriz de importância:  [2.         2.         2.23606798]
Cobertura:  6.23606797749979
Melhor conjunto:  [0, 1]


In [32]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Criar DataFrame
dados = pd.read_csv("dados.csv", sep=';')
print(dados)

# Vetorização
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(dados['review'])
y = dados['sentimentos']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo
model = LogisticRegression()
model.fit(X_train, y_train)

# Avaliação
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Função para fazer predição em uma nova frase
def predict_sentiment(text):
    text_vector = vectorizer.transform([text])
    prediction = model.predict_proba(text_vector)
    return prediction[0][1]  # Retorna a probabilidade de ser positivo

# Testando a função
new_review = "O filme é muito lento e chato. Não consegui me conectar com a história em nenhum momento."
print("Probabilidade de sentimento positivo:", predict_sentiment(new_review))

                                               review  sentimentos
0   Este filme foi absolutamente incrível! A trama...            1
1   Um desperdício total de tempo. O roteiro é fra...            0
2   O roteiro foi bem escrito e os atores entregar...            1
3   Não gostei nada da trama. Foi muito previsível...            0
4   Uma obra-prima cinematográfica que captura a e...            1
..                                                ...          ...
93         Totalmente decepcionante. Não vale a pena.            0
94   Não esperava isso, impossível parar de assistir.            1
95           Viciante, o enrendo te prende até o fim.            1
96                                 Uma bela porcaria.            0
97  As cenas são muito belas. Filme mais lindo que...            1

[98 rows x 2 columns]
Accuracy: 0.95
              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       1.00      0.93      0.96      

In [33]:
# Define a função para limpar a frase
def clean_text(sentence):
    sentence = sentence.strip().lower()
    sentence = re.sub(r'\s\s+', ' ', sentence)
    sentence = re.sub(r'[.,\/#!$%\^&\*;:{}=\-_`~()@?]', '', sentence)
    sentence = unicodedata.normalize('NFD', sentence)
    sentence = re.sub(r'[\u0300-\u036f]', '', sentence)
    words = sentence.split()
    filtered_words = [word for word in words if len(word) >= 3]
    return filtered_words

# Define a função para vetorizar a frase
def vectorize_sentences(sentence1, sentence2):
    sentence1 = clean_text(sentence1)
    sentence2 = clean_text(sentence2)
    words = list(set(sentence1 + sentence2))
    vector1 = np.zeros(len(words))
    vector2 = np.zeros(len(words))
    for i, word in enumerate(words):
        vector1[i] = sentence1.count(word)
        vector2[i] = sentence2.count(word)
    return vector1, vector2

# Define a função de similaridade
def similarity(vector1, vector2):
    return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    
def predict_sentiment(text):
    text_vector = vectorizer.transform([text])
    prediction = model.predict_proba(text_vector)
    return prediction[0][1]  # Retorna a probabilidade de ser positivo

class LimeExplainerSentences:
    def __init__(self, sigma=0.2, num_samples=1000, K=5, alpha=0.1, predict=predict_sentiment, p =10):
        self.sigma = sigma
        self.num_samples = num_samples
        self.K = K
        self.alpha = alpha
        self.predict = predict
        self.p = p
        
    # Define a função de kernel 
    def kernel(self, x, z):
        np.set_printoptions(precision=self.p)
        distance = similarity(x, z) # Similaridade de cosseno
        weights = np.sqrt(np.exp(-(distance**2) / (self.sigma**2)))  # Kernel 
        return weights
        
    # Gera dados ao redor de x_line
    def samples(self, x):
        np.set_printoptions(precision=self.p)
        x_vec = clean_text(x)
        n = len(x_vec)
        sample_set = []
        for i in range(self.num_samples):
            sample = np.zeros(n, dtype=int)
            num_ones = np.random.randint(1, self.K)
            indices = np.random.choice(n, num_ones, replace=False)
            sample[indices] = 1
            sample_set.append(sample)
        return sample_set
    
    # Transforma um vetor em uma frase
    def sentences_samples(self, x, z_line):
        z = []
        for i in range(len(clean_text(x))):
            if z_line[i] == 1:
                z.append(clean_text(x)[i])
        return " ".join(z)

    # Define o vetor de pesos
    def LIME(self, x):
        np.set_printoptions(precision=self.p)
        Z_line = self.samples(x)
        Z = [self.sentences_samples(x, z_line) for z_line in Z_line]
        Z_pred = [predict_sentiment(z) for z in Z]
        pi_x = []
        for z in Z:
            x_vec, z_vec = vectorize_sentences(x,z)
            pi_x.append(self.kernel(x_vec, z_vec)) 
        lasso = Lasso(alpha=self.alpha)
        lasso.fit(Z_line, Z_pred, sample_weight=pi_x)
        w = lasso.coef_
        return w
    
    def K_top_indices(self, x):
        abs_valores = np.abs(w)

    # Gerar explicação
    def explain_instance(self, x):
        np.set_printoptions(precision=self.p)
        x_vec = clean_text(x)
        w = self.LIME(x)
        abs_valores = np.abs(w)
        indices = np.argsort(abs_valores)[::-1][:self.K]
        print(f"Frase: {x}")
        print(f"w: {w}")
        print("Palavras importantes:")
        for i, word in enumerate(x_vec):
            if i in indices:
                print(f"{word}: {w[i]}")    


In [34]:
# Separar as features e o alvo
X = dados['review']
y = dados['sentimentos']
instance_index = 15
instance = X.iloc[instance_index]
instance_label = y.iloc[instance_index]
print("Instância: ", instance)
print("Rótulo: ", instance_label)

LIME = LimeExplainerSentences(sigma=0.1, num_samples=1885, K=3, alpha=0.1**(45),  predict=predict_sentiment)
LIME.explain_instance(instance)

Instância:  Nunca vi algo tão ruim na minha vida. A produção foi amadora e a história, entediante.
Rótulo:  0
Frase: Nunca vi algo tão ruim na minha vida. A produção foi amadora e a história, entediante.
w: [-0.0251157749  0.0040456922  0.0072241833 -0.1665830075 -0.0224641267
 -0.0251929131  0.0072716779  0.1450980535 -0.0252221873  0.0072159527
 -0.0252440058]
Palavras importantes:
ruim: -0.16658300748884525
foi: 0.14509805345196392
entediante: -0.025244005783576745


c:\Users\Isas_\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.362e-02, tolerance: 8.745e-04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
